# SSCN and PointNet Layer Implementation Assignment

In [1]:
#include <bits/stdc++.h>
using namespace std;

SSCN uses the preprocessing method used in [Volumetric and Multi-View CNNs for Object Classification on 3D Data](https://arxiv.org/abs/1604.03265)

We are going to use a simpler method that doesn't require another CNN, where we voxelize the given .off files and project them from multiple angles to get a 2d image
we can use.

### Download binvox and dataset

In [3]:
!test -f binvox || wget https://www.patrickmin.com/binvox/linux64/binvox

In [4]:
!chmod +x binvox

In [5]:
!./download_dataset.sh > /dev/null

### Runs script to voxelize `.off` file
`/dev/null` pipe is to stop it from breaking stdout, if the program is not working, remove it to debug

In [2]:
!./voxelize.sh ModelNet10/bed/train/bed_0009.off > /dev/null

## Parse voxel file into C++ memory

In [7]:
typedef unsigned char byte;

In [9]:
int read_binvox(string filespec){
  ifstream *input = new ifstream(filespec.c_str(), ios::in | ios::binary);

  // read header
  string line;
  *input >> line;  // #binvox
  if (line.compare("#binvox") != 0) {
    cout << "Error: first line reads [" << line << "] instead of [#binvox]" << endl;
    delete input;
    return 0;
  }
  int version;
  *input >> version;
  cout << "reading binvox version " << version << endl;

  int depth, height, width;
  depth = -1;
  int done = 0;
  while(input->good() && !done) {
    *input >> line;
    if (line.compare("data") == 0) done = 1;
    else if (line.compare("dim") == 0) {
      *input >> depth >> height >> width;
    }
    else {
      cout << "  unrecognized keyword [" << line << "], skipping" << endl;
      char c;
      do {  // skip until end of line
        c = input->get();
      } while(input->good() && (c != '\n'));

    }
  }
  if (!done) {
    cout << "  error reading header" << endl;
    return 0;
  }
  if (depth == -1) {
    cout << "  missing dimensions in header" << endl;
    return 0;
  }

  int size = width * height * depth;
  byte *voxels = new byte[size];
  if (!voxels) {
    cout << "  error allocating memory" << endl;
    return 0;
  }

  // read voxel data
  byte value;
  byte count;
  int index = 0;
  int end_index = 0;
  int nr_voxels = 0; 
  input->unsetf(ios::skipws);  // need to read every byte now (!)
  *input >> value;  // read the linefeed char

  while((end_index < size) && input->good()) {
    *input >> value >> count;
    if (input->good()) {
      end_index = index + count;
      if (end_index > size) return 0;
      for(int i=index; i < end_index; i++) voxels[i] = value;
      
      if (value) nr_voxels += count;
      index = end_index;
    }  // if file still ok
    
  }  // while
  input->close();
  cout << "  read " << nr_voxels << " voxels" << endl;
  return 1;
}


In [13]:
string file = "ModelNet10/bed/train/bed_0009.binvox";
read_binvox(file);

reading binvox version 1
  unrecognized keyword [translate], skipping
  unrecognized keyword [scale], skipping
  read 342548 voxels


In [ ]:
int foo = 1